In [1]:
print("Hello World :)")

Hello World :)


In [2]:
get_ipython().system('pip install db-dtypes')

In [3]:
import os
import pandas as pd
from google.cloud import bigquery
from google.oauth2 import service_account
from google.cloud.bigquery import magics
import matplotlib.pyplot as plt
import numpy as np

In [4]:
# Constants
PRED_WEEK = 40

In [5]:
BIGQUERY_PROJECT = 'ironhacks-data'
bigquery_client = bigquery.Client(project=BIGQUERY_PROJECT)

In [6]:
query_get_tables = """
SELECT * 
FROM `ironhacks-data.ironhacks_competition.INFORMATION_SCHEMA.TABLES`
"""
query_job = bigquery_client.query(query_get_tables)
query_job.to_dataframe().head()

,table_catalog,table_schema,table_name,table_type,is_insertable_into,is_typed,creation_time,base_table_catalog,base_table_schema,base_table_name,snapshot_time_ms,ddl,default_collation_name,upsert_stream_apply_watermark
0,ironhacks-data,ironhacks_competition,wage_data,BASE TABLE,YES,NO,2022-11-29 17:00:10.578000+00:00,None,None,None,NaT,CREATE TABLE `ironhacks-data.ironhacks_competi...,NULL,NaT
1,ironhacks-data,ironhacks_competition,prediction_list,BASE TABLE,YES,NO,2022-11-29 17:00:10.722000+00:00,None,None,None,NaT,CREATE TABLE `ironhacks-data.ironhacks_competi...,NULL,NaT
2,ironhacks-data,ironhacks_competition,unemployment_data,BASE TABLE,YES,NO,2022-11-29 17:00:14.588000+00:00,None,None,None,NaT,CREATE TABLE `ironhacks-data.ironhacks_competi...,NULL,NaT


In [7]:
query = """
SELECT *
FROM `ironhacks-data.ironhacks_competition.unemployment_data`
"""

In [8]:
query_job = bigquery_client.query(query)
unemployment_data = query_job.to_dataframe()
unemployment_data.head(2)

,uu_id,timeperiod,week_number,countyfips,tract,tract_name,total_claims,edu_8th_or_less,edu_grades_9_11,edu_hs_grad_equiv,...,gender_female,gender_male,gender_na,race_amerindian,race_asian,race_black,race_noanswer,race_hawaiiannative,race_other,race_white
0,f013068de98db1470bd986137a0c6d23,20220416,16,18003,900,"Census Tract 9, Allen County, Indiana",22,0,<NA>,14,...,<NA>,<NA>,0,0,0,11,0,0,<NA>,<NA>
1,21957d5517323845818d87623589e1ba,20220319,12,18089,10400,"Census Tract 104, Lake County, Indiana",111,0,<NA>,108,...,76,35,0,0,0,<NA>,<NA>,0,0,0


In [9]:
query = """
SELECT *
FROM `ironhacks-data.ironhacks_competition.prediction_list`
"""

In [10]:
query_job = bigquery_client.query(query)
prediction_list_data = query_job.to_dataframe()
prediction_list_data.head(2)

,uu_id,week_number
0,5bf51fc2e162d6faf9e3cf79e4198378,44
1,420b44cc7e3f55d738df565421e59941,44


In [11]:
query = """
SELECT *
FROM `ironhacks-data.ironhacks_competition.wage_data`
"""

In [12]:
query_job = bigquery_client.query(query)
wage_data = query_job.to_dataframe()
wage_data.head(2)

,uu_id,countyfips,tract,tract_name,average_wage
0,585f8731c2255d6b3f817a31180848b9,18177,200,"Census Tract 2, Wayne County, Indiana",6612.00
1,8c9d2aa90948679972a9382aadcc6001,18177,900,"Census Tract 9, Wayne County, Indiana",9883.25


In [13]:
query = """
SELECT *
FROM `ironhacks-data.ironhacks_competition.unemployment_data`
WHERE uu_id IN (SELECT uu_id
                FROM `ironhacks-data.ironhacks_competition.prediction_list`
                WHERE week_number = (SELECT MAX(week_number) FROM `ironhacks-data.ironhacks_competition.prediction_list`))
ORDER BY uu_id,week_number
"""

In [14]:
query_job = bigquery_client.query(query)
relevant_unemployment_df = query_job.to_dataframe()
relevant_unemployment_df.head(2)

,uu_id,timeperiod,week_number,countyfips,tract,tract_name,total_claims,edu_8th_or_less,edu_grades_9_11,edu_hs_grad_equiv,...,gender_female,gender_male,gender_na,race_amerindian,race_asian,race_black,race_noanswer,race_hawaiiannative,race_other,race_white
0,001cd9ae23064d7f0fd3cd327c873d8d,20220730,31,18099,20102,"Census Tract 201.02, Marshall County, Indiana",34,<NA>,0,30,...,<NA>,<NA>,0,0,0,0,0,0,0,34
1,001cd9ae23064d7f0fd3cd327c873d8d,20220813,33,18099,20102,"Census Tract 201.02, Marshall County, Indiana",10,<NA>,0,<NA>,...,<NA>,<NA>,0,0,0,0,0,0,0,10


In [15]:
relevant_unemployment_df.head(3)

,uu_id,timeperiod,week_number,countyfips,tract,tract_name,total_claims,edu_8th_or_less,edu_grades_9_11,edu_hs_grad_equiv,...,gender_female,gender_male,gender_na,race_amerindian,race_asian,race_black,race_noanswer,race_hawaiiannative,race_other,race_white
0,001cd9ae23064d7f0fd3cd327c873d8d,20220730,31,18099,20102,"Census Tract 201.02, Marshall County, Indiana",34,<NA>,0,30,...,<NA>,<NA>,0,0,0,0,0,0,0,34
1,001cd9ae23064d7f0fd3cd327c873d8d,20220813,33,18099,20102,"Census Tract 201.02, Marshall County, Indiana",10,<NA>,0,<NA>,...,<NA>,<NA>,0,0,0,0,0,0,0,10
2,001cd9ae23064d7f0fd3cd327c873d8d,20220813,33,18099,20102,"Census Tract 201.02, Marshall County, Indiana",10,<NA>,0,<NA>,...,<NA>,<NA>,0,0,0,0,0,0,0,10


In [16]:
# Drop duplicate columns
relevant_unemployment_df.drop_duplicates(inplace=True)
relevant_unemployment_df.drop(['timeperiod', 'tract_name'], axis=1, inplace=True)

In [17]:
# Drop columns with excessive null values
# NOTE: Revisit this, these columns may still be useful, especially those that aren't missing too many values
relevant_unemployment_df.dropna(axis='columns', inplace=True)
relevant_unemployment_df = relevant_unemployment_df.convert_dtypes()
relevant_unemployment_df['countyfips'] = relevant_unemployment_df['countyfips'].astype(str)
relevant_unemployment_df['tract'] = relevant_unemployment_df['tract'].astype(str)
relevant_unemployment_df['real_data'] = True
relevant_unemployment_df = relevant_unemployment_df.convert_dtypes()

Add rows such that every uu_id has rows for weeks 1:max(week_number)

In [18]:
# Add Wage data
relevant_unemployment_df['wage_data'] = relevant_unemployment_df['uu_id'].map(wage_data.set_index('uu_id')['average_wage'])
relevant_unemployment_df.info()
bad_wage_uu_ids = list(set(relevant_unemployment_df.loc[relevant_unemployment_df['wage_data'].isna(),'uu_id']))

<class 'pandas.core.frame.DataFrame'>
Int64Index: 13977 entries, 0 to 16831
Data columns (total 10 columns):
 #   Column                  Non-Null Count  Dtype  
---  ------                  --------------  -----  
 0   uu_id                   13977 non-null  string 
 1   week_number             13977 non-null  Int64  
 2   countyfips              13977 non-null  string 
 3   tract                   13977 non-null  string 
 4   total_claims            13977 non-null  Int64  
 5   top_category_employer1  13977 non-null  string 
 6   top_category_employer2  13977 non-null  string 
 7   top_category_employer3  13977 non-null  string 
 8   real_data               13977 non-null  boolean
 9   wage_data               13977 non-null  float64
dtypes: Int64(2), boolean(1), float64(1), string(6)
memory usage: 1.1 MB


In [19]:
for uuid in set(relevant_unemployment_df["uu_id"]):
    num_tracts = 0
    tracts=[]
    for tract in set(relevant_unemployment_df.loc[relevant_unemployment_df["uu_id"]==uuid,'wage_data']):
        num_tracts += 1
        tracts.append(tract)
    if num_tracts > 1:
        print(uuid,num_tracts,tracts)
relevant_unemployment_df

,uu_id,week_number,countyfips,tract,total_claims,top_category_employer1,top_category_employer2,top_category_employer3,real_data,wage_data
0,001cd9ae23064d7f0fd3cd327c873d8d,31,18099,20102,34,31-33,23,81,True,10484.333333
1,001cd9ae23064d7f0fd3cd327c873d8d,33,18099,20102,10,31-33,23,81,True,10484.333333
3,001cd9ae23064d7f0fd3cd327c873d8d,34,18099,20102,25,62,31-33,23,True,10484.333333
5,001cd9ae23064d7f0fd3cd327c873d8d,35,18099,20102,10,31-33,61,23,True,10484.333333
7,001cd9ae23064d7f0fd3cd327c873d8d,36,18099,20102,10,31-33,23,81,True,10484.333333
...,...,...,...,...,...,...,...,...,...,...
16823,fec479d0202d6e1e3f051a9ee902ff5d,33,18089,43300,59,62,23,31-33,True,9908.312500
16825,fec479d0202d6e1e3f051a9ee902ff5d,34,18089,43300,34,23,62,61,True,9908.312500
16827,fec479d0202d6e1e3f051a9ee902ff5d,35,18089,43300,27,62,23,72,True,9908.312500
16829,fec479d0202d6e1e3f051a9ee902ff5d,36,18089,43300,28,23,62,72,True,9908.312500


In [20]:
import warnings
warnings.simplefilter(action='ignore', category=FutureWarning)

In [21]:
temp_df = pd.DataFrame(columns=list(relevant_unemployment_df.columns))
my_ids = list(set(relevant_unemployment_df['uu_id']))
for UUid in my_ids:
    ctyfips = None
    wage_data = None
    tract_vote = [None, 0]
    top_cat_1_vote = [None, 0]
    top_cat_2_vote = [None, 0]
    top_cat_3_vote = [None, 0]
    total_claims_avg = []
    total_claims_num = 0
    
    for week in range(1,PRED_WEEK):
        row = relevant_unemployment_df.loc[((relevant_unemployment_df.uu_id==UUid) 
                                      & (relevant_unemployment_df.week_number == week)),]
        if len(row) == 1:
            temp_df = pd.concat([temp_df, row])
            
            ### Store information about values to infer for rows without data
            ## Consistent within uu_id values
            ctyfips = row['countyfips'].values[0]
            wage_data = row['wage_data'].values[0]
            
            ## Inconsistent within uu_id values
            # Categorical 

            if ((tract_vote[0] == None) or ((tract_vote[0] != row['tract'].values[0]) and (tract_vote[1] == 1))):
                tract_vote = [row['tract'].values[0], 1]
            elif tract_vote[0] == row['tract'].values[0]:
                tract_vote[1] += 1
            else:
                tract_vote[1] -= 1
                
            if ((top_cat_1_vote[0] == None) or ((top_cat_1_vote[0] != row['top_category_employer1'].values[0]) and (top_cat_1_vote[1] == 1))):
                top_cat_1_vote = [row['top_category_employer1'].values[0], 1]
            elif top_cat_1_vote[0] == row['top_category_employer1'].values[0]:
                top_cat_1_vote[1] += 1
            else:
                top_cat_1_vote[1] -= 1
                
            if ((top_cat_2_vote[0] == None) or ((top_cat_2_vote[0] != row['top_category_employer2'].values[0]) and (top_cat_2_vote[1] == 1))):
                top_cat_2_vote = [row['top_category_employer2'].values[0], 1]
            elif top_cat_2_vote[0] == row['top_category_employer2'].values[0]:
                top_cat_2_vote[1] += 1
            else:
                top_cat_2_vote[1] -= 1
                
            if ((top_cat_3_vote[0] == None) or ((top_cat_3_vote[0] != row['top_category_employer3'].values[0]) and (top_cat_3_vote[1] == 1))):
                top_cat_3_vote = [row['top_category_employer3'].values[0], 1]
            elif top_cat_3_vote[0] == row['top_category_employer3'].values[0]:
                top_cat_3_vote[1] += 1
            else:
                top_cat_3_vote[1] -= 1
                
            # Numerical
            total_claims_avg.append(row['total_claims'].values[0])
            if len(total_claims_avg) > 0:
                total_claims_num = np.mean(total_claims_avg)

            continue
        temp_df = pd.concat([temp_df, pd.DataFrame.from_records([{'uu_id': UUid, 
                                                                  'week_number': week, 
                                                                  'countyfips': ctyfips, 
                                                                  'tract': tract_vote[0],
                                                                  'total_claims': total_claims_num, 
                                                                  'top_category_employer1': top_cat_1_vote[0],
                                                                  'top_category_employer2': top_cat_2_vote[0], 
                                                                  'top_category_employer3': top_cat_3_vote[0],
                                                                  'real_data': False, 
                                                                  'wage_data': wage_data}])])

In [22]:
temp_df.reset_index(inplace=True)
temp_df.drop('index', axis=1,inplace=True)
temp_df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 20475 entries, 0 to 20474
Data columns (total 10 columns):
 #   Column                  Non-Null Count  Dtype  
---  ------                  --------------  -----  
 0   uu_id                   20475 non-null  object 
 1   week_number             20475 non-null  Int64  
 2   countyfips              20263 non-null  object 
 3   tract                   20263 non-null  object 
 4   total_claims            20475 non-null  Float64
 5   top_category_employer1  20263 non-null  object 
 6   top_category_employer2  20263 non-null  object 
 7   top_category_employer3  20263 non-null  object 
 8   real_data               20475 non-null  object 
 9   wage_data               20263 non-null  object 
dtypes: Float64(1), Int64(1), object(8)
memory usage: 1.6+ MB


In [23]:
temp_df.head()

,uu_id,week_number,countyfips,tract,total_claims,top_category_employer1,top_category_employer2,top_category_employer3,real_data,wage_data
0,147837d751cb44500754e9620204f8d6,1,18091,40800,27.0,44-45,23,56,True,5554.0
1,147837d751cb44500754e9620204f8d6,2,18091,40800,39.0,56,44-45,23,True,5554.0
2,147837d751cb44500754e9620204f8d6,3,18091,40800,31.0,31-33,23,56,True,5554.0
3,147837d751cb44500754e9620204f8d6,4,18091,40800,32.333333,31-33,23,56,False,5554.0
4,147837d751cb44500754e9620204f8d6,5,18091,40800,28.0,23,31-33,92,True,5554.0


In [24]:
non_null_cols = ["countyfips", "tract", "top_category_employer1", "top_category_employer2", "top_category_employer3", "wage_data"]

In [25]:
for UUid2 in my_ids[::-1]:
    for week_num in range(PRED_WEEK-1,0,-1):
        row = temp_df.loc[((temp_df['uu_id'] == UUid2) & (temp_df["week_number"] == week_num)),]
        if row['countyfips'].values[0] == None:
            for col in non_null_cols:
                temp_df.loc[((temp_df['uu_id'] == UUid2) & (temp_df["week_number"] == week_num)), col] = temp_df.loc[((temp_df['uu_id'] == UUid2) & (temp_df["week_number"] == week_num+1)), col].values[0]

In [26]:
temp_df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 20475 entries, 0 to 20474
Data columns (total 10 columns):
 #   Column                  Non-Null Count  Dtype  
---  ------                  --------------  -----  
 0   uu_id                   20475 non-null  object 
 1   week_number             20475 non-null  Int64  
 2   countyfips              20475 non-null  object 
 3   tract                   20475 non-null  object 
 4   total_claims            20475 non-null  Float64
 5   top_category_employer1  20475 non-null  object 
 6   top_category_employer2  20475 non-null  object 
 7   top_category_employer3  20475 non-null  object 
 8   real_data               20475 non-null  object 
 9   wage_data               20475 non-null  object 
dtypes: Float64(1), Int64(1), object(8)
memory usage: 1.6+ MB


In [27]:
temp_df.head()

,uu_id,week_number,countyfips,tract,total_claims,top_category_employer1,top_category_employer2,top_category_employer3,real_data,wage_data
0,147837d751cb44500754e9620204f8d6,1,18091,40800,27.0,44-45,23,56,True,5554.0
1,147837d751cb44500754e9620204f8d6,2,18091,40800,39.0,56,44-45,23,True,5554.0
2,147837d751cb44500754e9620204f8d6,3,18091,40800,31.0,31-33,23,56,True,5554.0
3,147837d751cb44500754e9620204f8d6,4,18091,40800,32.333333,31-33,23,56,False,5554.0
4,147837d751cb44500754e9620204f8d6,5,18091,40800,28.0,23,31-33,92,True,5554.0


In [28]:
temp_df = temp_df.convert_dtypes()
temp_df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 20475 entries, 0 to 20474
Data columns (total 10 columns):
 #   Column                  Non-Null Count  Dtype  
---  ------                  --------------  -----  
 0   uu_id                   20475 non-null  string 
 1   week_number             20475 non-null  Int64  
 2   countyfips              20475 non-null  string 
 3   tract                   20475 non-null  string 
 4   total_claims            20475 non-null  Float64
 5   top_category_employer1  20475 non-null  string 
 6   top_category_employer2  20475 non-null  string 
 7   top_category_employer3  20475 non-null  string 
 8   real_data               20475 non-null  boolean
 9   wage_data               20475 non-null  Float64
dtypes: Float64(2), Int64(1), boolean(1), string(6)
memory usage: 1.5 MB


In [29]:
bad_wage_ids=set(temp_df.loc[temp_df['wage_data'].isna(),"uu_id"])

In [30]:
data = temp_df
data.rename(columns={"uu_id": "id", "week_number": "week", "countyfips": "fips", "top_category_employer1": "emp1"}, inplace=True)
data.rename(columns={"top_category_employer2": "emp2", "top_category_employer3": "emp3", "real_data": "real", "wage_data":"wages"}, inplace=True)
for col in ['fips', 'tract', 'emp1', 'emp2', 'emp3']:
    data[col] = pd.factorize(data[col])[0]

In [31]:
data.head()

,id,week,fips,tract,total_claims,emp1,emp2,emp3,real,wages
0,147837d751cb44500754e9620204f8d6,1,0,0,27.0,0,0,0,True,5554.0
1,147837d751cb44500754e9620204f8d6,2,0,0,39.0,1,1,1,True,5554.0
2,147837d751cb44500754e9620204f8d6,3,0,0,31.0,2,0,0,True,5554.0
3,147837d751cb44500754e9620204f8d6,4,0,0,32.333333,2,0,0,False,5554.0
4,147837d751cb44500754e9620204f8d6,5,0,0,28.0,3,2,2,True,5554.0


In [32]:
data.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 20475 entries, 0 to 20474
Data columns (total 10 columns):
 #   Column        Non-Null Count  Dtype  
---  ------        --------------  -----  
 0   id            20475 non-null  string 
 1   week          20475 non-null  Int64  
 2   fips          20475 non-null  int64  
 3   tract         20475 non-null  int64  
 4   total_claims  20475 non-null  Float64
 5   emp1          20475 non-null  int64  
 6   emp2          20475 non-null  int64  
 7   emp3          20475 non-null  int64  
 8   real          20475 non-null  boolean
 9   wages         20475 non-null  Float64
dtypes: Float64(2), Int64(1), boolean(1), int64(5), string(1)
memory usage: 1.5 MB


In [33]:
abs(data.corr()['total_claims'])

week            0.042895
fips            0.145811
tract           0.010931
total_claims    1.000000
emp1            0.037770
emp2            0.016371
emp3            0.068482
real            0.165613
wages           0.123782
Name: total_claims, dtype: float64

In [34]:
ids = list(set(data['id']))
weeks = list(set(data['week']))
test_weeks = weeks[PRED_WEEK-10:]

In [35]:
from sklearn.linear_model import LinearRegression

In [36]:
predictions = dict()
actual = dict()
for pred_id in ids[:1]:
    predictions[pred_id] = []
    actual[pred_id] = []
    
    for wk in test_weeks:
        # Train test split
        if id not in bad_wage_ids:
            train_x = data.loc[((data['week'] < wk) & (data["id"] == pred_id)), 
                               ["fips", "tract", "emp1", "emp2", "emp3", "wages"]]
            test_x = data.loc[((data['week'] == wk) & (data["id"] == pred_id)), 
                               ["fips", "tract", "emp1", "emp2", "emp3", "wages"]]
        else:
            train_x = data.loc[((data['week'] < wk) & (data["id"] == pred_id)), 
                               ["fips", "tract", "emp1", "emp2", "emp3"]]
            test_x = data.loc[((data['week'] == wk) & (data["id"] == pred_id)), 
                               ["fips", "tract", "emp1", "emp2", "emp3"]]
        train_y = data.loc[((data['week'] < wk) & (data["id"] == pred_id)), "total_claims"]
        test_y = data.loc[((data['week'] == wk) & (data["id"] == pred_id)), "total_claims"]
        actual[pred_id].append(test_y)
        
        
        ## Prediction
        # Linear Model
        reg = LinearRegression().fit(train_x, train_y)
        prediction = reg.predict(test_x)
        predictions[pred_id].append(prediction)

In [37]:
predictions

{'147837d751cb44500754e9620204f8d6': [array([22.10646139]),
  array([22.6943862]),
  array([23.12389207]),
  array([23.45140669]),
  array([23.70940304]),
  array([20.10629602]),
  array([21.88937015]),
  array([31.18169667]),
  array([30.65812722])]}

In [38]:
from sklearn.linear_model import LinearRegression

In [39]:
predictions = dict()
actual = dict()
for pred_id in ids[:1]:
    predictions[pred_id] = []
    actual[pred_id] = []
    
    for wk in test_weeks:
        # Train test split
        if id not in bad_wage_ids:
            train_x = data.loc[((data['week'] < wk) & (data["id"] == pred_id)), 
                               ["fips", "tract", "emp1", "emp2", "emp3", "wages"]]
            test_x = data.loc[((data['week'] == wk) & (data["id"] == pred_id)), 
                               ["fips", "tract", "emp1", "emp2", "emp3", "wages"]]
        else:
            train_x = data.loc[((data['week'] < wk) & (data["id"] == pred_id)), 
                               ["fips", "tract", "emp1", "emp2", "emp3"]]
            test_x = data.loc[((data['week'] == wk) & (data["id"] == pred_id)), 
                               ["fips", "tract", "emp1", "emp2", "emp3"]]
        train_y = data.loc[((data['week'] < wk) & (data["id"] == pred_id)), "total_claims"]
        test_y = data.loc[((data['week'] == wk) & (data["id"] == pred_id)), "total_claims"]
        actual[pred_id].append(test_y)
        
        
        ## Prediction
        # Linear Model
        reg = LinearRegression().fit(train_x, train_y)
        prediction = reg.predict(test_x)
        predictions[pred_id].append(prediction[0])

In [40]:
predictions

{'147837d751cb44500754e9620204f8d6': [22.10646138549395,
  22.6943862005158,
  23.123892072688,
  23.45140668522994,
  23.70940303873978,
  20.10629601662575,
  21.889370145487778,
  31.181696666600082,
  30.658127219454926]}

In [41]:
from sklearn.linear_model import LinearRegression

In [42]:
predictions = dict()
actual = dict()
for pred_id in ids[:1]:
    predictions[pred_id] = []
    actual[pred_id] = []
    
    for wk in test_weeks:
        # Train test split
        if id not in bad_wage_ids:
            train_x = data.loc[((data['week'] < wk) & (data["id"] == pred_id)), 
                               ["fips", "tract", "emp1", "emp2", "emp3", "wages"]]
            test_x = data.loc[((data['week'] == wk) & (data["id"] == pred_id)), 
                               ["fips", "tract", "emp1", "emp2", "emp3", "wages"]]
        else:
            train_x = data.loc[((data['week'] < wk) & (data["id"] == pred_id)), 
                               ["fips", "tract", "emp1", "emp2", "emp3"]]
            test_x = data.loc[((data['week'] == wk) & (data["id"] == pred_id)), 
                               ["fips", "tract", "emp1", "emp2", "emp3"]]
        train_y = data.loc[((data['week'] < wk) & (data["id"] == pred_id)), "total_claims"]
        test_y = data.loc[((data['week'] == wk) & (data["id"] == pred_id)), "total_claims"]
        actual[pred_id].append(test_y)
        print(train_x)
        
        ## Prediction
        # Linear Model
        reg = LinearRegression().fit(train_x, train_y)
        prediction = reg.predict(test_x)
        predictions[pred_id].append(prediction[0])

    fips  tract  emp1  emp2  emp3   wages
0      0      0     0     0     0  5554.0
1      0      0     1     1     1  5554.0
2      0      0     2     0     0  5554.0
3      0      0     2     0     0  5554.0
4      0      0     3     2     2  5554.0
5      0      0     3     2     3  5554.0
6      0      0     3     3     4  5554.0
7      0      0     3     3     3  5554.0
8      0      0     3     3     4  5554.0
9      0      0     0     0     5  5554.0
10     0      0     1     1     1  5554.0
11     0      0     3     4     0  5554.0
12     0      0     1     0     2  5554.0
13     0      0     2     4     6  5554.0
14     0      0     1     2     1  5554.0
15     0      0     3     2     1  5554.0
16     0      0     3     2     1  5554.0
17     0      0     3     2     1  5554.0
18     0      0     3     2     1  5554.0
19     0      0     1     1     4  5554.0
20     0      0     1     1     4  5554.0
21     0      0     0     5     4  5554.0
22     0      0     0     5     4 

In [43]:
from sklearn.linear_model import LinearRegression

In [44]:
predictions = dict()
actual = dict()
for pred_id in ids[:1]:
    predictions[pred_id] = []
    actual[pred_id] = []
    
    for wk in test_weeks:
        # Train test split
        if id not in bad_wage_ids:
            train_x = data.loc[((data['week'] < wk) & (data["id"] == pred_id)), 
                               ["fips", "tract", "emp1", "emp2", "emp3", "wages"]]
            test_x = data.loc[((data['week'] == wk) & (data["id"] == pred_id)), 
                               ["fips", "tract", "emp1", "emp2", "emp3", "wages"]]
        else:
            train_x = data.loc[((data['week'] < wk) & (data["id"] == pred_id)), 
                               ["fips", "tract", "emp1", "emp2", "emp3"]]
            test_x = data.loc[((data['week'] == wk) & (data["id"] == pred_id)), 
                               ["fips", "tract", "emp1", "emp2", "emp3"]]
        train_y = data.loc[((data['week'] < wk) & (data["id"] == pred_id)), "total_claims"]
        test_y = data.loc[((data['week'] == wk) & (data["id"] == pred_id)), "total_claims"]
        actual[pred_id].append(test_y)
        
        ## Prediction
        # Linear Model
        reg = LinearRegression().fit(train_x, train_y)
        prediction = reg.predict(test_x)
        predictions[pred_id].append(prediction[0])

In [45]:
actual

{'147837d751cb44500754e9620204f8d6': [30    25.882353
  Name: total_claims, dtype: Float64,
  31    25.882353
  Name: total_claims, dtype: Float64,
  32    25.882353
  Name: total_claims, dtype: Float64,
  33    25.882353
  Name: total_claims, dtype: Float64,
  34    25.882353
  Name: total_claims, dtype: Float64,
  35    21.0
  Name: total_claims, dtype: Float64,
  36    70.0
  Name: total_claims, dtype: Float64,
  37    27.947368
  Name: total_claims, dtype: Float64,
  38    27.947368
  Name: total_claims, dtype: Float64]}

In [46]:
from sklearn.linear_model import LinearRegression

In [47]:
predictions = dict()
actual = dict()
for pred_id in ids[:1]:
    predictions[pred_id] = []
    actual[pred_id] = []
    
    for wk in test_weeks:
        # Train test split
        if id not in bad_wage_ids:
            train_x = data.loc[((data['week'] < wk) & (data["id"] == pred_id)), 
                               ["fips", "tract", "emp1", "emp2", "emp3", "wages"]]
            test_x = data.loc[((data['week'] == wk) & (data["id"] == pred_id)), 
                               ["fips", "tract", "emp1", "emp2", "emp3", "wages"]]
        else:
            train_x = data.loc[((data['week'] < wk) & (data["id"] == pred_id)), 
                               ["fips", "tract", "emp1", "emp2", "emp3"]]
            test_x = data.loc[((data['week'] == wk) & (data["id"] == pred_id)), 
                               ["fips", "tract", "emp1", "emp2", "emp3"]]
        train_y = data.loc[((data['week'] < wk) & (data["id"] == pred_id)), "total_claims"]
        test_y = data.loc[((data['week'] == wk) & (data["id"] == pred_id)), "total_claims"]
        actual[pred_id].append(test_y['total_claims']).values[0]
        
        ## Prediction
        # Linear Model
        reg = LinearRegression().fit(train_x, train_y)
        prediction = reg.predict(test_x)
        predictions[pred_id].append(prediction[0])

KeyError: 'total_claims'

In [48]:
from sklearn.linear_model import LinearRegression

In [49]:
predictions = dict()
actual = dict()
for pred_id in ids[:1]:
    predictions[pred_id] = []
    actual[pred_id] = []
    
    for wk in test_weeks:
        # Train test split
        if id not in bad_wage_ids:
            train_x = data.loc[((data['week'] < wk) & (data["id"] == pred_id)), 
                               ["fips", "tract", "emp1", "emp2", "emp3", "wages"]]
            test_x = data.loc[((data['week'] == wk) & (data["id"] == pred_id)), 
                               ["fips", "tract", "emp1", "emp2", "emp3", "wages"]]
        else:
            train_x = data.loc[((data['week'] < wk) & (data["id"] == pred_id)), 
                               ["fips", "tract", "emp1", "emp2", "emp3"]]
            test_x = data.loc[((data['week'] == wk) & (data["id"] == pred_id)), 
                               ["fips", "tract", "emp1", "emp2", "emp3"]]
        train_y = data.loc[((data['week'] < wk) & (data["id"] == pred_id)), "total_claims"]
        test_y = data.loc[((data['week'] == wk) & (data["id"] == pred_id)), "total_claims"]
        actual[pred_id].append(test_y['total_claims'].values[0])
        
        ## Prediction
        # Linear Model
        reg = LinearRegression().fit(train_x, train_y)
        prediction = reg.predict(test_x)
        predictions[pred_id].append(prediction[0])

KeyError: 'total_claims'

In [50]:
from sklearn.linear_model import LinearRegression

In [51]:
predictions = dict()
actual = dict()
for pred_id in ids[:1]:
    predictions[pred_id] = []
    actual[pred_id] = []
    
    for wk in test_weeks:
        # Train test split
        if id not in bad_wage_ids:
            train_x = data.loc[((data['week'] < wk) & (data["id"] == pred_id)), 
                               ["fips", "tract", "emp1", "emp2", "emp3", "wages"]]
            test_x = data.loc[((data['week'] == wk) & (data["id"] == pred_id)), 
                               ["fips", "tract", "emp1", "emp2", "emp3", "wages"]]
        else:
            train_x = data.loc[((data['week'] < wk) & (data["id"] == pred_id)), 
                               ["fips", "tract", "emp1", "emp2", "emp3"]]
            test_x = data.loc[((data['week'] == wk) & (data["id"] == pred_id)), 
                               ["fips", "tract", "emp1", "emp2", "emp3"]]
        train_y = data.loc[((data['week'] < wk) & (data["id"] == pred_id)), "total_claims"]
        test_y = data.loc[((data['week'] == wk) & (data["id"] == pred_id)), "total_claims"]
        actual[pred_id].append(test_y)
        
        ## Prediction
        # Linear Model
        reg = LinearRegression().fit(train_x, train_y)
        prediction = reg.predict(test_x)
        predictions[pred_id].append(prediction[0])

In [52]:
predictions

{'147837d751cb44500754e9620204f8d6': [22.10646138549395,
  22.6943862005158,
  23.123892072688,
  23.45140668522994,
  23.70940303873978,
  20.10629601662575,
  21.889370145487778,
  31.181696666600082,
  30.658127219454926]}

In [53]:
actual

{'147837d751cb44500754e9620204f8d6': [30    25.882353
  Name: total_claims, dtype: Float64,
  31    25.882353
  Name: total_claims, dtype: Float64,
  32    25.882353
  Name: total_claims, dtype: Float64,
  33    25.882353
  Name: total_claims, dtype: Float64,
  34    25.882353
  Name: total_claims, dtype: Float64,
  35    21.0
  Name: total_claims, dtype: Float64,
  36    70.0
  Name: total_claims, dtype: Float64,
  37    27.947368
  Name: total_claims, dtype: Float64,
  38    27.947368
  Name: total_claims, dtype: Float64]}

In [54]:
from sklearn.linear_model import LinearRegression

In [55]:
predictions = dict()
actual = dict()
for pred_id in ids[:1]:
    predictions[pred_id] = []
    actual[pred_id] = []
    
    for wk in test_weeks:
        # Train test split
        if id not in bad_wage_ids:
            train_x = data.loc[((data['week'] < wk) & (data["id"] == pred_id)), 
                               ["fips", "tract", "emp1", "emp2", "emp3", "wages"]]
            test_x = data.loc[((data['week'] == wk) & (data["id"] == pred_id)), 
                               ["fips", "tract", "emp1", "emp2", "emp3", "wages"]]
        else:
            train_x = data.loc[((data['week'] < wk) & (data["id"] == pred_id)), 
                               ["fips", "tract", "emp1", "emp2", "emp3"]]
            test_x = data.loc[((data['week'] == wk) & (data["id"] == pred_id)), 
                               ["fips", "tract", "emp1", "emp2", "emp3"]]
        train_y = data.loc[((data['week'] < wk) & (data["id"] == pred_id)), "total_claims"]
        test_y = data.loc[((data['week'] == wk) & (data["id"] == pred_id)), "total_claims"]
        actual[pred_id].append(test_y[0])
        
        ## Prediction
        # Linear Model
        reg = LinearRegression().fit(train_x, train_y)
        prediction = reg.predict(test_x)
        predictions[pred_id].append(prediction[0])

KeyError: 0

In [56]:
predictions

{'147837d751cb44500754e9620204f8d6': []}

In [57]:
from sklearn.linear_model import LinearRegression

In [58]:
predictions = dict()
actual = dict()
for pred_id in ids[:1]:
    predictions[pred_id] = []
    actual[pred_id] = []
    
    for wk in test_weeks:
        # Train test split
        if id not in bad_wage_ids:
            train_x = data.loc[((data['week'] < wk) & (data["id"] == pred_id)), 
                               ["fips", "tract", "emp1", "emp2", "emp3", "wages"]]
            test_x = data.loc[((data['week'] == wk) & (data["id"] == pred_id)), 
                               ["fips", "tract", "emp1", "emp2", "emp3", "wages"]]
        else:
            train_x = data.loc[((data['week'] < wk) & (data["id"] == pred_id)), 
                               ["fips", "tract", "emp1", "emp2", "emp3"]]
            test_x = data.loc[((data['week'] == wk) & (data["id"] == pred_id)), 
                               ["fips", "tract", "emp1", "emp2", "emp3"]]
        train_y = data.loc[((data['week'] < wk) & (data["id"] == pred_id)), "total_claims"]
        test_y = data.loc[((data['week'] == wk) & (data["id"] == pred_id)), "total_claims"]
        actual[pred_id].append(test_y.values[0])
        
        ## Prediction
        # Linear Model
        reg = LinearRegression().fit(train_x, train_y)
        prediction = reg.predict(test_x)
        predictions[pred_id].append(prediction[0])

In [59]:
predictions

{'147837d751cb44500754e9620204f8d6': [22.10646138549395,
  22.6943862005158,
  23.123892072688,
  23.45140668522994,
  23.70940303873978,
  20.10629601662575,
  21.889370145487778,
  31.181696666600082,
  30.658127219454926]}

In [60]:
actual

{'147837d751cb44500754e9620204f8d6': [25.88235294117647,
  25.88235294117647,
  25.88235294117647,
  25.88235294117647,
  25.88235294117647,
  21.0,
  70.0,
  27.94736842105263,
  27.94736842105263]}

In [61]:
from sklearn.linear_model import LinearRegression

In [62]:
predictions = dict()
actual = dict()
for pred_id in ids[:1]:
    predictions[pred_id] = []
    actual[pred_id] = []
    
    for wk in test_weeks:
        # Train test split
        if id not in bad_wage_ids:
            train_x = data.loc[((data['week'] < wk) & (data["id"] == pred_id)), 
                               ["fips", "tract", "emp1", "emp2", "emp3", "wages"]]
            test_x = data.loc[((data['week'] == wk) & (data["id"] == pred_id)), 
                               ["fips", "tract", "emp1", "emp2", "emp3", "wages"]]
        else:
            train_x = data.loc[((data['week'] < wk) & (data["id"] == pred_id)), 
                               ["fips", "tract", "emp1", "emp2", "emp3"]]
            print(train_x
            test_x = data.loc[((data['week'] == wk) & (data["id"] == pred_id)), 
                               ["fips", "tract", "emp1", "emp2", "emp3"]]
        train_y = data.loc[((data['week'] < wk) & (data["id"] == pred_id)), "total_claims"]
        test_y = data.loc[((data['week'] == wk) & (data["id"] == pred_id)), "total_claims"]
        actual[pred_id].append(test_y.values[0])
        
        ## Prediction
        # Linear Model
        reg = LinearRegression().fit(train_x, train_y)
        prediction = reg.predict(test_x)
        predictions[pred_id].append(prediction[0])

SyntaxError: invalid syntax (4234595879.py, line 18)

In [63]:
from sklearn.linear_model import LinearRegression

In [64]:
predictions = dict()
actual = dict()
for pred_id in ids[:1]:
    predictions[pred_id] = []
    actual[pred_id] = []
    
    for wk in test_weeks:
        # Train test split
        if id not in bad_wage_ids:
            train_x = data.loc[((data['week'] < wk) & (data["id"] == pred_id)), 
                               ["fips", "tract", "emp1", "emp2", "emp3", "wages"]]
            test_x = data.loc[((data['week'] == wk) & (data["id"] == pred_id)), 
                               ["fips", "tract", "emp1", "emp2", "emp3", "wages"]]
        else:
            train_x = data.loc[((data['week'] < wk) & (data["id"] == pred_id)), 
                               ["fips", "tract", "emp1", "emp2", "emp3"]]
            print(train_x)
            test_x = data.loc[((data['week'] == wk) & (data["id"] == pred_id)), 
                               ["fips", "tract", "emp1", "emp2", "emp3"]]
        train_y = data.loc[((data['week'] < wk) & (data["id"] == pred_id)), "total_claims"]
        test_y = data.loc[((data['week'] == wk) & (data["id"] == pred_id)), "total_claims"]
        actual[pred_id].append(test_y.values[0])
        
        ## Prediction
        # Linear Model
        reg = LinearRegression().fit(train_x, train_y)
        prediction = reg.predict(test_x)
        predictions[pred_id].append(prediction[0])

In [65]:
from sklearn.linear_model import LinearRegression

In [66]:
predictions = dict()
actual = dict()
for pred_id in ids[:1]:
    predictions[pred_id] = []
    actual[pred_id] = []
    
    for wk in test_weeks:
        # Train test split
        if id not in bad_wage_ids:
            train_x = data.loc[((data['week'] < wk) & (data["id"] == pred_id)), 
                               ["fips", "tract", "emp1", "emp2", "emp3", "wages"]]
            test_x = data.loc[((data['week'] == wk) & (data["id"] == pred_id)), 
                               ["fips", "tract", "emp1", "emp2", "emp3", "wages"]]
        else:
            train_x = data.loc[((data['week'] < wk) & (data["id"] == pred_id)), 
                               ["fips", "tract", "emp1", "emp2", "emp3"]]
            test_x = data.loc[((data['week'] == wk) & (data["id"] == pred_id)), 
                               ["fips", "tract", "emp1", "emp2", "emp3"]]
        print(train_x)
        train_y = data.loc[((data['week'] < wk) & (data["id"] == pred_id)), "total_claims"]
        test_y = data.loc[((data['week'] == wk) & (data["id"] == pred_id)), "total_claims"]
        actual[pred_id].append(test_y.values[0])
        
        ## Prediction
        # Linear Model
        reg = LinearRegression().fit(train_x, train_y)
        prediction = reg.predict(test_x)
        predictions[pred_id].append(prediction[0])

    fips  tract  emp1  emp2  emp3   wages
0      0      0     0     0     0  5554.0
1      0      0     1     1     1  5554.0
2      0      0     2     0     0  5554.0
3      0      0     2     0     0  5554.0
4      0      0     3     2     2  5554.0
5      0      0     3     2     3  5554.0
6      0      0     3     3     4  5554.0
7      0      0     3     3     3  5554.0
8      0      0     3     3     4  5554.0
9      0      0     0     0     5  5554.0
10     0      0     1     1     1  5554.0
11     0      0     3     4     0  5554.0
12     0      0     1     0     2  5554.0
13     0      0     2     4     6  5554.0
14     0      0     1     2     1  5554.0
15     0      0     3     2     1  5554.0
16     0      0     3     2     1  5554.0
17     0      0     3     2     1  5554.0
18     0      0     3     2     1  5554.0
19     0      0     1     1     4  5554.0
20     0      0     1     1     4  5554.0
21     0      0     0     5     4  5554.0
22     0      0     0     5     4 

In [67]:
from sklearn.linear_model import LinearRegression

In [68]:
predictions = dict()
actual = dict()
for pred_id in ids[:1]:
    predictions[pred_id] = []
    actual[pred_id] = []
    
    for wk in test_weeks:
        # Train test split
        if id not in bad_wage_ids:
            train_x = data.loc[((data['week'] < wk) & (data["id"] == pred_id)), 
                               ["fips", "tract", "emp1", "emp2", "emp3", "wages"]]
            test_x = data.loc[((data['week'] == wk) & (data["id"] == pred_id)), 
                               ["fips", "tract", "emp1", "emp2", "emp3", "wages"]]
        else:
            train_x = data.loc[((data['week'] < wk) & (data["id"] == pred_id)), 
                               ["fips", "tract", "emp1", "emp2", "emp3"]]
            test_x = data.loc[((data['week'] == wk) & (data["id"] == pred_id)), 
                               ["fips", "tract", "emp1", "emp2", "emp3"]]
        print(train_y)
        train_y = data.loc[((data['week'] < wk) & (data["id"] == pred_id)), "total_claims"]
        test_y = data.loc[((data['week'] == wk) & (data["id"] == pred_id)), "total_claims"]
        actual[pred_id].append(test_y.values[0])
        
        ## Prediction
        # Linear Model
        reg = LinearRegression().fit(train_x, train_y)
        prediction = reg.predict(test_x)
        predictions[pred_id].append(prediction[0])

0          27.0
1          39.0
2          31.0
3     32.333333
4          28.0
5          21.0
6          20.0
7          32.0
8          23.0
9          29.0
10         33.0
11         18.0
12         46.0
13         30.0
14         18.0
15    28.214286
16    28.214286
17    28.214286
18    28.214286
19         23.0
20    27.866667
21         12.0
22       26.875
23       26.875
24       26.875
25       26.875
26       26.875
27       26.875
28       26.875
29         10.0
30    25.882353
31    25.882353
32    25.882353
33    25.882353
34    25.882353
35         21.0
36         70.0
37    27.947368
Name: total_claims, dtype: Float64
0          27.0
1          39.0
2          31.0
3     32.333333
4          28.0
5          21.0
6          20.0
7          32.0
8          23.0
9          29.0
10         33.0
11         18.0
12         46.0
13         30.0
14         18.0
15    28.214286
16    28.214286
17    28.214286
18    28.214286
19         23.0
20    27.866667
21         12.0
22   

In [69]:
from sklearn.linear_model import LinearRegression

In [70]:
predictions = dict()
actual = dict()
for pred_id in ids[:1]:
    predictions[pred_id] = []
    actual[pred_id] = []
    
    for wk in test_weeks:
        # Train test split
        if id not in bad_wage_ids:
            train_x = data.loc[((data['week'] < wk) & (data["id"] == pred_id)), 
                               ["fips", "tract", "emp1", "emp2", "emp3", "wages"]]
            test_x = data.loc[((data['week'] == wk) & (data["id"] == pred_id)), 
                               ["fips", "tract", "emp1", "emp2", "emp3", "wages"]]
        else:
            train_x = data.loc[((data['week'] < wk) & (data["id"] == pred_id)), 
                               ["fips", "tract", "emp1", "emp2", "emp3"]]
            test_x = data.loc[((data['week'] == wk) & (data["id"] == pred_id)), 
                               ["fips", "tract", "emp1", "emp2", "emp3"]]
        print(pred_id)
        train_y = data.loc[((data['week'] < wk) & (data["id"] == pred_id)), "total_claims"]
        test_y = data.loc[((data['week'] == wk) & (data["id"] == pred_id)), "total_claims"]
        actual[pred_id].append(test_y.values[0])
        
        ## Prediction
        # Linear Model
        reg = LinearRegression().fit(train_x, train_y)
        prediction = reg.predict(test_x)
        predictions[pred_id].append(prediction[0])

147837d751cb44500754e9620204f8d6
147837d751cb44500754e9620204f8d6
147837d751cb44500754e9620204f8d6
147837d751cb44500754e9620204f8d6
147837d751cb44500754e9620204f8d6
147837d751cb44500754e9620204f8d6
147837d751cb44500754e9620204f8d6
147837d751cb44500754e9620204f8d6
147837d751cb44500754e9620204f8d6


In [71]:
temp_df

,id,week,fips,tract,total_claims,emp1,emp2,emp3,real,wages
0,147837d751cb44500754e9620204f8d6,1,0,0,27.0,0,0,0,True,5554.0
1,147837d751cb44500754e9620204f8d6,2,0,0,39.0,1,1,1,True,5554.0
2,147837d751cb44500754e9620204f8d6,3,0,0,31.0,2,0,0,True,5554.0
3,147837d751cb44500754e9620204f8d6,4,0,0,32.333333,2,0,0,False,5554.0
4,147837d751cb44500754e9620204f8d6,5,0,0,28.0,3,2,2,True,5554.0
...,...,...,...,...,...,...,...,...,...,...
20470,ae2f6787bb01d859420365f1ab55a3c1,35,13,424,27.0,1,0,3,True,11012.583333
20471,ae2f6787bb01d859420365f1ab55a3c1,36,13,424,29.0,1,0,5,True,11012.583333
20472,ae2f6787bb01d859420365f1ab55a3c1,37,13,424,27.0,8,8,0,True,11012.583333
20473,ae2f6787bb01d859420365f1ab55a3c1,38,13,424,33.742857,8,0,0,False,11012.583333


In [72]:
temp_df.head(39)

,id,week,fips,tract,total_claims,emp1,emp2,emp3,real,wages
0,147837d751cb44500754e9620204f8d6,1,0,0,27.0,0,0,0,True,5554.0
1,147837d751cb44500754e9620204f8d6,2,0,0,39.0,1,1,1,True,5554.0
2,147837d751cb44500754e9620204f8d6,3,0,0,31.0,2,0,0,True,5554.0
3,147837d751cb44500754e9620204f8d6,4,0,0,32.333333,2,0,0,False,5554.0
4,147837d751cb44500754e9620204f8d6,5,0,0,28.0,3,2,2,True,5554.0
5,147837d751cb44500754e9620204f8d6,6,0,0,21.0,3,2,3,True,5554.0
6,147837d751cb44500754e9620204f8d6,7,0,0,20.0,3,3,4,True,5554.0
7,147837d751cb44500754e9620204f8d6,8,0,0,32.0,3,3,3,True,5554.0
8,147837d751cb44500754e9620204f8d6,9,0,0,23.0,3,3,4,True,5554.0
9,147837d751cb44500754e9620204f8d6,10,0,0,29.0,0,0,5,True,5554.0


In [73]:
from sklearn.linear_model import LinearRegression
from sklearn.metrics import mean_squared_error
from sklearn.metrics import mean_absolute_error

In [74]:
predictions = dict()
actual = dict()
MSE = dict()
MAE = dict()
for pred_id in ids[:1]:
    predictions[pred_id] = []
    actual[pred_id] = []
    MSE[pred_id] = []
    MAE[pred_id] = []
    
    for wk in test_weeks:
        # Train test split
        if id not in bad_wage_ids:
            train_x = data.loc[((data['week'] < wk) & (data["id"] == pred_id)), 
                               ["fips", "tract", "emp1", "emp2", "emp3", "wages"]]
            test_x = data.loc[((data['week'] == wk) & (data["id"] == pred_id)), 
                               ["fips", "tract", "emp1", "emp2", "emp3", "wages"]]
        else:
            train_x = data.loc[((data['week'] < wk) & (data["id"] == pred_id)), 
                               ["fips", "tract", "emp1", "emp2", "emp3"]]
            test_x = data.loc[((data['week'] == wk) & (data["id"] == pred_id)), 
                               ["fips", "tract", "emp1", "emp2", "emp3"]]
        train_y = data.loc[((data['week'] < wk) & (data["id"] == pred_id)), "total_claims"]
        test_y = data.loc[((data['week'] == wk) & (data["id"] == pred_id)), "total_claims"]
        actual[pred_id].append(test_y.values[0])
        
        ## Prediction
        # Linear Model
        reg = LinearRegression().fit(train_x, train_y)
        prediction = reg.predict(test_x)
        predictions[pred_id].append(prediction[0])
    
    # Error metrics
    MSE[pred_id].append(mean_squared_error(actual[pred_id], predictions[pred_id]))
    MAE[pred_id].append(mean_absolute_error(actual[pred_id], predictions[pred_id]))

In [75]:
predictions

{'147837d751cb44500754e9620204f8d6': [22.10646138549395,
  22.6943862005158,
  23.123892072688,
  23.45140668522994,
  23.70940303873978,
  20.10629601662575,
  21.889370145487778,
  31.181696666600082,
  30.658127219454926]}

In [76]:
actual

{'147837d751cb44500754e9620204f8d6': [25.88235294117647,
  25.88235294117647,
  25.88235294117647,
  25.88235294117647,
  25.88235294117647,
  21.0,
  70.0,
  27.94736842105263,
  27.94736842105263]}

In [77]:
MSE

{'147837d751cb44500754e9620204f8d6': [263.9890345101622]}

In [78]:
MAE

{'147837d751cb44500754e9620204f8d6': [7.697292911672345]}

In [79]:
from sklearn.linear_model import LinearRegression
from sklearn.metrics import mean_squared_error
from sklearn.metrics import mean_absolute_error

In [80]:
predictions = dict()
actual = dict()
MSE = dict()
MAE = dict()
for pred_id in ids:
    predictions[pred_id] = []
    actual[pred_id] = []
    MSE[pred_id] = []
    MAE[pred_id] = []
    
    for wk in test_weeks:
        # Train test split
        if id not in bad_wage_ids:
            train_x = data.loc[((data['week'] < wk) & (data["id"] == pred_id)), 
                               ["fips", "tract", "emp1", "emp2", "emp3", "wages"]]
            test_x = data.loc[((data['week'] == wk) & (data["id"] == pred_id)), 
                               ["fips", "tract", "emp1", "emp2", "emp3", "wages"]]
        else:
            train_x = data.loc[((data['week'] < wk) & (data["id"] == pred_id)), 
                               ["fips", "tract", "emp1", "emp2", "emp3"]]
            test_x = data.loc[((data['week'] == wk) & (data["id"] == pred_id)), 
                               ["fips", "tract", "emp1", "emp2", "emp3"]]
        train_y = data.loc[((data['week'] < wk) & (data["id"] == pred_id)), "total_claims"]
        test_y = data.loc[((data['week'] == wk) & (data["id"] == pred_id)), "total_claims"]
        actual[pred_id].append(test_y.values[0])
        
        ## Prediction
        # Linear Model
        reg = LinearRegression().fit(train_x, train_y)
        prediction = reg.predict(test_x)
        predictions[pred_id].append(prediction[0])
    
    # Error metrics
    MSE[pred_id].append(mean_squared_error(actual[pred_id], predictions[pred_id]))
    MAE[pred_id].append(mean_absolute_error(actual[pred_id], predictions[pred_id]))

In [81]:
from sklearn.linear_model import LinearRegression
from sklearn.metrics import mean_squared_error
from sklearn.metrics import mean_absolute_error

In [82]:
predictions = dict()
actual = dict()
MSE = dict()
MAE = dict()
for pred_id in ids:
    predictions[pred_id] = []
    actual[pred_id] = []
    MSE[pred_id] = []
    MAE[pred_id] = []
    
    for wk in test_weeks:
        # Train test split
        if id not in bad_wage_ids:
            train_x = data.loc[((data['week'] < wk) & (data["id"] == pred_id)), 
                               ["fips", "tract", "emp1", "emp2", "emp3", "wages"]]
            test_x = data.loc[((data['week'] == wk) & (data["id"] == pred_id)), 
                               ["fips", "tract", "emp1", "emp2", "emp3", "wages"]]
        else:
            train_x = data.loc[((data['week'] < wk) & (data["id"] == pred_id)), 
                               ["fips", "tract", "emp1", "emp2", "emp3"]]
            test_x = data.loc[((data['week'] == wk) & (data["id"] == pred_id)), 
                               ["fips", "tract", "emp1", "emp2", "emp3"]]
        train_y = data.loc[((data['week'] < wk) & (data["id"] == pred_id)), "total_claims"]
        test_y = data.loc[((data['week'] == wk) & (data["id"] == pred_id)), "total_claims"]
        actual[pred_id].append(test_y.values[0])
        
        ## Prediction
        # Linear Model
        print(pred_id)
        reg = LinearRegression().fit(train_x, train_y)
        prediction = reg.predict(test_x)
        predictions[pred_id].append(prediction[0])
    
    # Error metrics
    MSE[pred_id].append(mean_squared_error(actual[pred_id], predictions[pred_id]))
    MAE[pred_id].append(mean_absolute_error(actual[pred_id], predictions[pred_id]))

147837d751cb44500754e9620204f8d6
147837d751cb44500754e9620204f8d6
147837d751cb44500754e9620204f8d6
147837d751cb44500754e9620204f8d6
147837d751cb44500754e9620204f8d6
147837d751cb44500754e9620204f8d6
147837d751cb44500754e9620204f8d6
147837d751cb44500754e9620204f8d6
147837d751cb44500754e9620204f8d6
58313733000358ab97030c3688616a8b
58313733000358ab97030c3688616a8b
58313733000358ab97030c3688616a8b
58313733000358ab97030c3688616a8b
58313733000358ab97030c3688616a8b
58313733000358ab97030c3688616a8b
58313733000358ab97030c3688616a8b
58313733000358ab97030c3688616a8b
58313733000358ab97030c3688616a8b
5846c7b267dc6e4b00863277e3582f80
5846c7b267dc6e4b00863277e3582f80
5846c7b267dc6e4b00863277e3582f80
5846c7b267dc6e4b00863277e3582f80
5846c7b267dc6e4b00863277e3582f80
5846c7b267dc6e4b00863277e3582f80
5846c7b267dc6e4b00863277e3582f80
5846c7b267dc6e4b00863277e3582f80
5846c7b267dc6e4b00863277e3582f80
7b1f33221592c5bc981035ecfcd359f9
7b1f33221592c5bc981035ecfcd359f9
7b1f33221592c5bc981035ecfcd359f9
7b1f332215

6f483941cceef95c97d28b4f904a0d57
6f483941cceef95c97d28b4f904a0d57
6f483941cceef95c97d28b4f904a0d57
6f483941cceef95c97d28b4f904a0d57
6f483941cceef95c97d28b4f904a0d57
6f483941cceef95c97d28b4f904a0d57
d44e7a9d13d3730892b6ffff9af4d595
d44e7a9d13d3730892b6ffff9af4d595
d44e7a9d13d3730892b6ffff9af4d595
d44e7a9d13d3730892b6ffff9af4d595
d44e7a9d13d3730892b6ffff9af4d595
d44e7a9d13d3730892b6ffff9af4d595
d44e7a9d13d3730892b6ffff9af4d595
d44e7a9d13d3730892b6ffff9af4d595
d44e7a9d13d3730892b6ffff9af4d595
5a9758f65f001b6432ff31ff64a459d7
5a9758f65f001b6432ff31ff64a459d7
5a9758f65f001b6432ff31ff64a459d7
5a9758f65f001b6432ff31ff64a459d7
5a9758f65f001b6432ff31ff64a459d7
5a9758f65f001b6432ff31ff64a459d7
5a9758f65f001b6432ff31ff64a459d7
5a9758f65f001b6432ff31ff64a459d7
5a9758f65f001b6432ff31ff64a459d7
e8b3b95e93a6dc7dbb90f4e72e7ac065
e8b3b95e93a6dc7dbb90f4e72e7ac065
e8b3b95e93a6dc7dbb90f4e72e7ac065
e8b3b95e93a6dc7dbb90f4e72e7ac065
e8b3b95e93a6dc7dbb90f4e72e7ac065
e8b3b95e93a6dc7dbb90f4e72e7ac065
e8b3b95e93

a0875049afbd1cb4735e07c03fb8baae
a0875049afbd1cb4735e07c03fb8baae
a0875049afbd1cb4735e07c03fb8baae
4765514a954c2fc3eb7c1993662e60e9
4765514a954c2fc3eb7c1993662e60e9
4765514a954c2fc3eb7c1993662e60e9
4765514a954c2fc3eb7c1993662e60e9
4765514a954c2fc3eb7c1993662e60e9
4765514a954c2fc3eb7c1993662e60e9
4765514a954c2fc3eb7c1993662e60e9
4765514a954c2fc3eb7c1993662e60e9
4765514a954c2fc3eb7c1993662e60e9
447d5984bb8a8179bff002ccd1bba162
447d5984bb8a8179bff002ccd1bba162
447d5984bb8a8179bff002ccd1bba162
447d5984bb8a8179bff002ccd1bba162
447d5984bb8a8179bff002ccd1bba162
447d5984bb8a8179bff002ccd1bba162
447d5984bb8a8179bff002ccd1bba162
447d5984bb8a8179bff002ccd1bba162
447d5984bb8a8179bff002ccd1bba162
99fc3cb44cdaf10cc69506d7c92b6466
99fc3cb44cdaf10cc69506d7c92b6466
99fc3cb44cdaf10cc69506d7c92b6466
99fc3cb44cdaf10cc69506d7c92b6466
99fc3cb44cdaf10cc69506d7c92b6466
99fc3cb44cdaf10cc69506d7c92b6466
99fc3cb44cdaf10cc69506d7c92b6466
99fc3cb44cdaf10cc69506d7c92b6466
99fc3cb44cdaf10cc69506d7c92b6466
747f8bc2b0

7953cef088684d6a82637fe6a9e7e7a8
7953cef088684d6a82637fe6a9e7e7a8
7953cef088684d6a82637fe6a9e7e7a8
7953cef088684d6a82637fe6a9e7e7a8
7953cef088684d6a82637fe6a9e7e7a8
7953cef088684d6a82637fe6a9e7e7a8
7953cef088684d6a82637fe6a9e7e7a8
7953cef088684d6a82637fe6a9e7e7a8
7953cef088684d6a82637fe6a9e7e7a8
bf8df4bed5c89ab2aca38cc649b8038e
bf8df4bed5c89ab2aca38cc649b8038e
bf8df4bed5c89ab2aca38cc649b8038e
bf8df4bed5c89ab2aca38cc649b8038e
bf8df4bed5c89ab2aca38cc649b8038e
bf8df4bed5c89ab2aca38cc649b8038e
bf8df4bed5c89ab2aca38cc649b8038e
bf8df4bed5c89ab2aca38cc649b8038e
bf8df4bed5c89ab2aca38cc649b8038e
258c76c5552e3c876c3e24e9658c0205
258c76c5552e3c876c3e24e9658c0205
258c76c5552e3c876c3e24e9658c0205
258c76c5552e3c876c3e24e9658c0205
258c76c5552e3c876c3e24e9658c0205
258c76c5552e3c876c3e24e9658c0205
258c76c5552e3c876c3e24e9658c0205
258c76c5552e3c876c3e24e9658c0205
258c76c5552e3c876c3e24e9658c0205
98e63567a2f4907c9cb2637247651530
98e63567a2f4907c9cb2637247651530
98e63567a2f4907c9cb2637247651530
98e63567a2

5c43cbf8f0d860e91a9160706ea9b7d8
5c43cbf8f0d860e91a9160706ea9b7d8
5c43cbf8f0d860e91a9160706ea9b7d8
5c43cbf8f0d860e91a9160706ea9b7d8
5c43cbf8f0d860e91a9160706ea9b7d8
5c43cbf8f0d860e91a9160706ea9b7d8
ee9da2a37f88c1f0e922757286c26270
ee9da2a37f88c1f0e922757286c26270
ee9da2a37f88c1f0e922757286c26270
ee9da2a37f88c1f0e922757286c26270
ee9da2a37f88c1f0e922757286c26270
ee9da2a37f88c1f0e922757286c26270
ee9da2a37f88c1f0e922757286c26270
ee9da2a37f88c1f0e922757286c26270
ee9da2a37f88c1f0e922757286c26270
2ddb832dc6e28aba32224f559c5f0b63
2ddb832dc6e28aba32224f559c5f0b63
2ddb832dc6e28aba32224f559c5f0b63
2ddb832dc6e28aba32224f559c5f0b63
2ddb832dc6e28aba32224f559c5f0b63
2ddb832dc6e28aba32224f559c5f0b63
2ddb832dc6e28aba32224f559c5f0b63
2ddb832dc6e28aba32224f559c5f0b63
2ddb832dc6e28aba32224f559c5f0b63
03ba3a9a63be1a5423457246f2846292
03ba3a9a63be1a5423457246f2846292
03ba3a9a63be1a5423457246f2846292
03ba3a9a63be1a5423457246f2846292
03ba3a9a63be1a5423457246f2846292
03ba3a9a63be1a5423457246f2846292
03ba3a9a63

...

184e8a1febbf737c9865b40ac43d92ee
184e8a1febbf737c9865b40ac43d92ee
7bff91298a7f07d1cdcc7cb13c49eb41
7bff91298a7f07d1cdcc7cb13c49eb41
7bff91298a7f07d1cdcc7cb13c49eb41
7bff91298a7f07d1cdcc7cb13c49eb41
7bff91298a7f07d1cdcc7cb13c49eb41
7bff91298a7f07d1cdcc7cb13c49eb41
7bff91298a7f07d1cdcc7cb13c49eb41
7bff91298a7f07d1cdcc7cb13c49eb41
7bff91298a7f07d1cdcc7cb13c49eb41
1f53903c302f96fd0e8a36d02364bbca
1f53903c302f96fd0e8a36d02364bbca
1f53903c302f96fd0e8a36d02364bbca
1f53903c302f96fd0e8a36d02364bbca
1f53903c302f96fd0e8a36d02364bbca
1f53903c302f96fd0e8a36d02364bbca
1f53903c302f96fd0e8a36d02364bbca
1f53903c302f96fd0e8a36d02364bbca
1f53903c302f96fd0e8a36d02364bbca
420b44cc7e3f55d738df565421e59941
420b44cc7e3f55d738df565421e59941
420b44cc7e3f55d738df565421e59941
420b44cc7e3f55d738df565421e59941


420b44cc7e3f55d738df565421e59941
420b44cc7e3f55d738df565421e59941
420b44cc7e3f55d738df565421e59941
420b44cc7e3f55d738df565421e59941
420b44cc7e3f55d738df565421e59941
695e104bd84215f8226793190ee20baf
695e104bd84215f8226793190ee20baf
695e104bd84215f8226793190ee20baf
695e104bd84215f8226793190ee20baf
695e104bd84215f8226793190ee20baf
695e104bd84215f8226793190ee20baf
695e104bd84215f8226793190ee20baf
695e104bd84215f8226793190ee20baf
695e104bd84215f8226793190ee20baf
dfa86215568c869a2e94ce3e5fcf1661
dfa86215568c869a2e94ce3e5fcf1661


dfa86215568c869a2e94ce3e5fcf1661
dfa86215568c869a2e94ce3e5fcf1661
dfa86215568c869a2e94ce3e5fcf1661
dfa86215568c869a2e94ce3e5fcf1661
dfa86215568c869a2e94ce3e5fcf1661
dfa86215568c869a2e94ce3e5fcf1661
dfa86215568c869a2e94ce3e5fcf1661
37495d17e82f7df326bfc2c4c090f7b7
37495d17e82f7df326bfc2c4c090f7b7
37495d17e82f7df326bfc2c4c090f7b7
37495d17e82f7df326bfc2c4c090f7b7
37495d17e82f7df326bfc2c4c090f7b7
37495d17e82f7df326bfc2c4c090f7b7
37495d17e82f7df326bfc2c4c090f7b7
37495d17e82f7df326bfc2c4c090f7b7
37495d17e82f7df326bfc2c4c090f7b7
67bbd99237964244c2ad3d0531394291
67bbd99237964244c2ad3d0531394291
67bbd99237964244c2ad3d0531394291
67bbd99237964244c2ad3d0531394291
67bbd99237964244c2ad3d0531394291
67bbd99237964244c2ad3d0531394291
67bbd99237964244c2ad3d0531394291
67bbd99237964244c2ad3d0531394291
67bbd99237964244c2ad3d0531394291
a514bda8b5262164df92063a50786753
a514bda8b5262164df92063a50786753
a514bda8b5262164df92063a50786753


a514bda8b5262164df92063a50786753
a514bda8b5262164df92063a50786753
a514bda8b5262164df92063a50786753
a514bda8b5262164df92063a50786753
a514bda8b5262164df92063a50786753
a514bda8b5262164df92063a50786753
983badfd7b568728e39a2344a9006078
983badfd7b568728e39a2344a9006078
983badfd7b568728e39a2344a9006078
983badfd7b568728e39a2344a9006078
983badfd7b568728e39a2344a9006078
983badfd7b568728e39a2344a9006078
983badfd7b568728e39a2344a9006078
983badfd7b568728e39a2344a9006078
983badfd7b568728e39a2344a9006078
179a3fb4630992ff1f9112970d61f00f
179a3fb4630992ff1f9112970d61f00f
179a3fb4630992ff1f9112970d61f00f
179a3fb4630992ff1f9112970d61f00f
179a3fb4630992ff1f9112970d61f00f
179a3fb4630992ff1f9112970d61f00f
179a3fb4630992ff1f9112970d61f00f
179a3fb4630992ff1f9112970d61f00f
179a3fb4630992ff1f9112970d61f00f
bcf10911a28373ba22a779145eaa9942
bcf10911a28373ba22a779145eaa9942
bcf10911a28373ba22a779145eaa9942
bcf10911a28373ba22a779145eaa9942
bcf10911a28373ba22a779145eaa9942


bcf10911a28373ba22a779145eaa9942
bcf10911a28373ba22a779145eaa9942
bcf10911a28373ba22a779145eaa9942
bcf10911a28373ba22a779145eaa9942
ae2f6787bb01d859420365f1ab55a3c1
ae2f6787bb01d859420365f1ab55a3c1
ae2f6787bb01d859420365f1ab55a3c1
ae2f6787bb01d859420365f1ab55a3c1
ae2f6787bb01d859420365f1ab55a3c1
ae2f6787bb01d859420365f1ab55a3c1
ae2f6787bb01d859420365f1ab55a3c1
ae2f6787bb01d859420365f1ab55a3c1
ae2f6787bb01d859420365f1ab55a3c1


In [83]:
bad_ids

NameError: name 'bad_ids' is not defined

In [84]:
bad_wage_ids

set()

In [85]:
train_x

,fips,tract,emp1,emp2,emp3,wages
20436,13,424,1,0,7,11012.583333
20437,13,424,8,0,0,11012.583333
20438,13,424,2,0,5,11012.583333
20439,13,424,2,0,5,11012.583333
20440,13,424,11,0,7,11012.583333
20441,13,424,3,10,4,11012.583333
20442,13,424,5,10,1,11012.583333
20443,13,424,5,0,5,11012.583333
20444,13,424,8,0,17,11012.583333
20445,13,424,0,10,7,11012.583333


In [86]:
from sklearn.linear_model import LinearRegression
from sklearn.metrics import mean_squared_error
from sklearn.metrics import mean_absolute_error

In [87]:
predictions = dict()
actual = dict()
MSE = dict()
MAE = dict()
for pred_id in ids:
    predictions[pred_id] = []
    actual[pred_id] = []
    MSE[pred_id] = []
    MAE[pred_id] = []
    
    for wk in test_weeks:
        # Train test split
        if pred_id not in bad_wage_ids:
            train_x = data.loc[((data['week'] < wk) & (data["id"] == pred_id)), 
                               ["fips", "tract", "emp1", "emp2", "emp3", "wages"]]
            test_x = data.loc[((data['week'] == wk) & (data["id"] == pred_id)), 
                               ["fips", "tract", "emp1", "emp2", "emp3", "wages"]]
        else:
            train_x = data.loc[((data['week'] < wk) & (data["id"] == pred_id)), 
                               ["fips", "tract", "emp1", "emp2", "emp3"]]
            test_x = data.loc[((data['week'] == wk) & (data["id"] == pred_id)), 
                               ["fips", "tract", "emp1", "emp2", "emp3"]]
        train_y = data.loc[((data['week'] < wk) & (data["id"] == pred_id)), "total_claims"]
        test_y = data.loc[((data['week'] == wk) & (data["id"] == pred_id)), "total_claims"]
        actual[pred_id].append(test_y.values[0])
        
        ## Prediction
        # Linear Model
        print(pred_id)
        reg = LinearRegression().fit(train_x, train_y)
        prediction = reg.predict(test_x)
        predictions[pred_id].append(prediction[0])
    
    # Error metrics
    MSE[pred_id].append(mean_squared_error(actual[pred_id], predictions[pred_id]))
    MAE[pred_id].append(mean_absolute_error(actual[pred_id], predictions[pred_id]))

147837d751cb44500754e9620204f8d6
147837d751cb44500754e9620204f8d6
147837d751cb44500754e9620204f8d6
147837d751cb44500754e9620204f8d6
147837d751cb44500754e9620204f8d6
147837d751cb44500754e9620204f8d6
147837d751cb44500754e9620204f8d6
147837d751cb44500754e9620204f8d6
147837d751cb44500754e9620204f8d6
58313733000358ab97030c3688616a8b
58313733000358ab97030c3688616a8b
58313733000358ab97030c3688616a8b
58313733000358ab97030c3688616a8b
58313733000358ab97030c3688616a8b
58313733000358ab97030c3688616a8b
58313733000358ab97030c3688616a8b
58313733000358ab97030c3688616a8b
58313733000358ab97030c3688616a8b
5846c7b267dc6e4b00863277e3582f80
5846c7b267dc6e4b00863277e3582f80
5846c7b267dc6e4b00863277e3582f80


5846c7b267dc6e4b00863277e3582f80
5846c7b267dc6e4b00863277e3582f80
5846c7b267dc6e4b00863277e3582f80
5846c7b267dc6e4b00863277e3582f80
5846c7b267dc6e4b00863277e3582f80
5846c7b267dc6e4b00863277e3582f80
7b1f33221592c5bc981035ecfcd359f9
7b1f33221592c5bc981035ecfcd359f9
7b1f33221592c5bc981035ecfcd359f9
7b1f33221592c5bc981035ecfcd359f9
7b1f33221592c5bc981035ecfcd359f9
7b1f33221592c5bc981035ecfcd359f9
7b1f33221592c5bc981035ecfcd359f9
7b1f33221592c5bc981035ecfcd359f9
7b1f33221592c5bc981035ecfcd359f9
6f483941cceef95c97d28b4f904a0d57
6f483941cceef95c97d28b4f904a0d57
6f483941cceef95c97d28b4f904a0d57
6f483941cceef95c97d28b4f904a0d57


6f483941cceef95c97d28b4f904a0d57
6f483941cceef95c97d28b4f904a0d57
6f483941cceef95c97d28b4f904a0d57
6f483941cceef95c97d28b4f904a0d57
6f483941cceef95c97d28b4f904a0d57
d44e7a9d13d3730892b6ffff9af4d595
d44e7a9d13d3730892b6ffff9af4d595
d44e7a9d13d3730892b6ffff9af4d595
d44e7a9d13d3730892b6ffff9af4d595
d44e7a9d13d3730892b6ffff9af4d595
d44e7a9d13d3730892b6ffff9af4d595
d44e7a9d13d3730892b6ffff9af4d595
d44e7a9d13d3730892b6ffff9af4d595


d44e7a9d13d3730892b6ffff9af4d595
5a9758f65f001b6432ff31ff64a459d7
5a9758f65f001b6432ff31ff64a459d7
5a9758f65f001b6432ff31ff64a459d7
5a9758f65f001b6432ff31ff64a459d7
5a9758f65f001b6432ff31ff64a459d7
5a9758f65f001b6432ff31ff64a459d7
5a9758f65f001b6432ff31ff64a459d7
5a9758f65f001b6432ff31ff64a459d7
5a9758f65f001b6432ff31ff64a459d7
e8b3b95e93a6dc7dbb90f4e72e7ac065
e8b3b95e93a6dc7dbb90f4e72e7ac065
e8b3b95e93a6dc7dbb90f4e72e7ac065
e8b3b95e93a6dc7dbb90f4e72e7ac065
e8b3b95e93a6dc7dbb90f4e72e7ac065
e8b3b95e93a6dc7dbb90f4e72e7ac065
e8b3b95e93a6dc7dbb90f4e72e7ac065
e8b3b95e93a6dc7dbb90f4e72e7ac065
e8b3b95e93a6dc7dbb90f4e72e7ac065
a0875049afbd1cb4735e07c03fb8baae
a0875049afbd1cb4735e07c03fb8baae
a0875049afbd1cb4735e07c03fb8baae
a0875049afbd1cb4735e07c03fb8baae
a0875049afbd1cb4735e07c03fb8baae
a0875049afbd1cb4735e07c03fb8baae
a0875049afbd1cb4735e07c03fb8baae
a0875049afbd1cb4735e07c03fb8baae
a0875049afbd1cb4735e07c03fb8baae
4765514a954c2fc3eb7c1993662e60e9


4765514a954c2fc3eb7c1993662e60e9
4765514a954c2fc3eb7c1993662e60e9
4765514a954c2fc3eb7c1993662e60e9
4765514a954c2fc3eb7c1993662e60e9
4765514a954c2fc3eb7c1993662e60e9
4765514a954c2fc3eb7c1993662e60e9
4765514a954c2fc3eb7c1993662e60e9
4765514a954c2fc3eb7c1993662e60e9
447d5984bb8a8179bff002ccd1bba162
447d5984bb8a8179bff002ccd1bba162
447d5984bb8a8179bff002ccd1bba162
447d5984bb8a8179bff002ccd1bba162
447d5984bb8a8179bff002ccd1bba162
447d5984bb8a8179bff002ccd1bba162
447d5984bb8a8179bff002ccd1bba162


...

74c5fdc6aac736fa7cf9b5d45c501a03
74c5fdc6aac736fa7cf9b5d45c501a03
74c5fdc6aac736fa7cf9b5d45c501a03
74c5fdc6aac736fa7cf9b5d45c501a03
74c5fdc6aac736fa7cf9b5d45c501a03
74c5fdc6aac736fa7cf9b5d45c501a03
74c5fdc6aac736fa7cf9b5d45c501a03
93c603c6ec0389076b9f852f9250a231
93c603c6ec0389076b9f852f9250a231
93c603c6ec0389076b9f852f9250a231
93c603c6ec0389076b9f852f9250a231
93c603c6ec0389076b9f852f9250a231
93c603c6ec0389076b9f852f9250a231
93c603c6ec0389076b9f852f9250a231
93c603c6ec0389076b9f852f9250a231
93c603c6ec0389076b9f852f9250a231
f35320206f3dd1ffc411e4ac127caf92
f35320206f3dd1ffc411e4ac127caf92
f35320206f3dd1ffc411e4ac127caf92
f35320206f3dd1ffc411e4ac127caf92
f35320206f3dd1ffc411e4ac127caf92
f35320206f3dd1ffc411e4ac127caf92
f35320206f3dd1ffc411e4ac127caf92
f35320206f3dd1ffc411e4ac127caf92
f35320206f3dd1ffc411e4ac127caf92
50088293eaca82f8c05e74c7aa38733f
50088293eaca82f8c05e74c7aa38733f
50088293eaca82f8c05e74c7aa38733f
50088293eaca82f8c05e74c7aa38733f
50088293eaca82f8c05e74c7aa38733f
50088293ea

0ad94f09274e2c9cb0ef5cb77eb334b4
0ad94f09274e2c9cb0ef5cb77eb334b4
0ad94f09274e2c9cb0ef5cb77eb334b4
0ad94f09274e2c9cb0ef5cb77eb334b4
0ad94f09274e2c9cb0ef5cb77eb334b4
bff474e03c775cd4d326df7044b6911e
bff474e03c775cd4d326df7044b6911e
bff474e03c775cd4d326df7044b6911e
bff474e03c775cd4d326df7044b6911e
bff474e03c775cd4d326df7044b6911e
bff474e03c775cd4d326df7044b6911e
bff474e03c775cd4d326df7044b6911e
bff474e03c775cd4d326df7044b6911e
bff474e03c775cd4d326df7044b6911e
d3bf31d4fda5dd60330943a904ac5d5a
d3bf31d4fda5dd60330943a904ac5d5a
d3bf31d4fda5dd60330943a904ac5d5a
d3bf31d4fda5dd60330943a904ac5d5a
d3bf31d4fda5dd60330943a904ac5d5a
d3bf31d4fda5dd60330943a904ac5d5a
d3bf31d4fda5dd60330943a904ac5d5a
d3bf31d4fda5dd60330943a904ac5d5a
d3bf31d4fda5dd60330943a904ac5d5a
5530c743ed9acfbd5d9ce30aa9b5a394
5530c743ed9acfbd5d9ce30aa9b5a394
5530c743ed9acfbd5d9ce30aa9b5a394
5530c743ed9acfbd5d9ce30aa9b5a394
5530c743ed9acfbd5d9ce30aa9b5a394
5530c743ed9acfbd5d9ce30aa9b5a394
5530c743ed9acfbd5d9ce30aa9b5a394
5530c743ed

184e8a1febbf737c9865b40ac43d92ee
184e8a1febbf737c9865b40ac43d92ee
184e8a1febbf737c9865b40ac43d92ee
7bff91298a7f07d1cdcc7cb13c49eb41
7bff91298a7f07d1cdcc7cb13c49eb41
7bff91298a7f07d1cdcc7cb13c49eb41
7bff91298a7f07d1cdcc7cb13c49eb41
7bff91298a7f07d1cdcc7cb13c49eb41
7bff91298a7f07d1cdcc7cb13c49eb41
7bff91298a7f07d1cdcc7cb13c49eb41
7bff91298a7f07d1cdcc7cb13c49eb41
7bff91298a7f07d1cdcc7cb13c49eb41
1f53903c302f96fd0e8a36d02364bbca
1f53903c302f96fd0e8a36d02364bbca
1f53903c302f96fd0e8a36d02364bbca
1f53903c302f96fd0e8a36d02364bbca
1f53903c302f96fd0e8a36d02364bbca
1f53903c302f96fd0e8a36d02364bbca
1f53903c302f96fd0e8a36d02364bbca
1f53903c302f96fd0e8a36d02364bbca
1f53903c302f96fd0e8a36d02364bbca
420b44cc7e3f55d738df565421e59941
420b44cc7e3f55d738df565421e59941
420b44cc7e3f55d738df565421e59941
420b44cc7e3f55d738df565421e59941
420b44cc7e3f55d738df565421e59941
420b44cc7e3f55d738df565421e59941
420b44cc7e3f55d738df565421e59941
420b44cc7e3f55d738df565421e59941
420b44cc7e3f55d738df565421e59941
695e104bd8

dfa86215568c869a2e94ce3e5fcf1661
dfa86215568c869a2e94ce3e5fcf1661
dfa86215568c869a2e94ce3e5fcf1661
dfa86215568c869a2e94ce3e5fcf1661
dfa86215568c869a2e94ce3e5fcf1661
dfa86215568c869a2e94ce3e5fcf1661
dfa86215568c869a2e94ce3e5fcf1661
dfa86215568c869a2e94ce3e5fcf1661
37495d17e82f7df326bfc2c4c090f7b7
37495d17e82f7df326bfc2c4c090f7b7
37495d17e82f7df326bfc2c4c090f7b7
37495d17e82f7df326bfc2c4c090f7b7
37495d17e82f7df326bfc2c4c090f7b7
37495d17e82f7df326bfc2c4c090f7b7
37495d17e82f7df326bfc2c4c090f7b7
37495d17e82f7df326bfc2c4c090f7b7
37495d17e82f7df326bfc2c4c090f7b7
67bbd99237964244c2ad3d0531394291
67bbd99237964244c2ad3d0531394291
67bbd99237964244c2ad3d0531394291
67bbd99237964244c2ad3d0531394291
67bbd99237964244c2ad3d0531394291
67bbd99237964244c2ad3d0531394291
67bbd99237964244c2ad3d0531394291
67bbd99237964244c2ad3d0531394291
67bbd99237964244c2ad3d0531394291
a514bda8b5262164df92063a50786753
a514bda8b5262164df92063a50786753
a514bda8b5262164df92063a50786753
a514bda8b5262164df92063a50786753
a514bda8b5

983badfd7b568728e39a2344a9006078
983badfd7b568728e39a2344a9006078
983badfd7b568728e39a2344a9006078
983badfd7b568728e39a2344a9006078
179a3fb4630992ff1f9112970d61f00f
179a3fb4630992ff1f9112970d61f00f
179a3fb4630992ff1f9112970d61f00f
179a3fb4630992ff1f9112970d61f00f
179a3fb4630992ff1f9112970d61f00f
179a3fb4630992ff1f9112970d61f00f
179a3fb4630992ff1f9112970d61f00f
179a3fb4630992ff1f9112970d61f00f
179a3fb4630992ff1f9112970d61f00f
bcf10911a28373ba22a779145eaa9942
bcf10911a28373ba22a779145eaa9942
bcf10911a28373ba22a779145eaa9942
bcf10911a28373ba22a779145eaa9942
bcf10911a28373ba22a779145eaa9942
bcf10911a28373ba22a779145eaa9942
bcf10911a28373ba22a779145eaa9942
bcf10911a28373ba22a779145eaa9942
bcf10911a28373ba22a779145eaa9942
ae2f6787bb01d859420365f1ab55a3c1
ae2f6787bb01d859420365f1ab55a3c1
ae2f6787bb01d859420365f1ab55a3c1
ae2f6787bb01d859420365f1ab55a3c1
ae2f6787bb01d859420365f1ab55a3c1
ae2f6787bb01d859420365f1ab55a3c1
ae2f6787bb01d859420365f1ab55a3c1
ae2f6787bb01d859420365f1ab55a3c1
ae2f6787bb

In [88]:
from sklearn.linear_model import LinearRegression
from sklearn.metrics import mean_squared_error
from sklearn.metrics import mean_absolute_error

In [89]:
predictions = dict()
actual = dict()
MSE = dict()
MAE = dict()
for pred_id in ids:
    predictions[pred_id] = []
    actual[pred_id] = []
    MSE[pred_id] = []
    MAE[pred_id] = []
    
    for wk in test_weeks:
        # Train test split
        if pred_id not in bad_wage_ids:
            train_x = data.loc[((data['week'] < wk) & (data["id"] == pred_id)), 
                               ["fips", "tract", "emp1", "emp2", "emp3", "wages"]]
            test_x = data.loc[((data['week'] == wk) & (data["id"] == pred_id)), 
                               ["fips", "tract", "emp1", "emp2", "emp3", "wages"]]
        else:
            train_x = data.loc[((data['week'] < wk) & (data["id"] == pred_id)), 
                               ["fips", "tract", "emp1", "emp2", "emp3"]]
            test_x = data.loc[((data['week'] == wk) & (data["id"] == pred_id)), 
                               ["fips", "tract", "emp1", "emp2", "emp3"]]
        train_y = data.loc[((data['week'] < wk) & (data["id"] == pred_id)), "total_claims"]
        test_y = data.loc[((data['week'] == wk) & (data["id"] == pred_id)), "total_claims"]
        actual[pred_id].append(test_y.values[0])
        
        ## Prediction
        # Linear Model
        reg = LinearRegression().fit(train_x, train_y)
        prediction = reg.predict(test_x)
        predictions[pred_id].append(prediction[0])
    
    # Error metrics
    MSE[pred_id].append(mean_squared_error(actual[pred_id], predictions[pred_id]))
    MAE[pred_id].append(mean_absolute_error(actual[pred_id], predictions[pred_id]))

In [90]:
MSE

{'147837d751cb44500754e9620204f8d6': [263.9890345101622],
 '58313733000358ab97030c3688616a8b': [53.249253498223005],
 '5846c7b267dc6e4b00863277e3582f80': [17.402469346134296],
 '7b1f33221592c5bc981035ecfcd359f9': [2068.378615569107],
 '6f483941cceef95c97d28b4f904a0d57': [19.14204541869776],
 'd44e7a9d13d3730892b6ffff9af4d595': [593.2474157680158],
 '5a9758f65f001b6432ff31ff64a459d7': [13.532008874440157],
 'e8b3b95e93a6dc7dbb90f4e72e7ac065': [11.768214824446552],
 'a0875049afbd1cb4735e07c03fb8baae': [30.67665482758258],
 '4765514a954c2fc3eb7c1993662e60e9': [4.09043955528531],
 '447d5984bb8a8179bff002ccd1bba162': [88.28313526781753],
 '99fc3cb44cdaf10cc69506d7c92b6466': [224.91513551752217],
 '747f8bc2b0c8c0a04d29caa4cfe327d2': [1.4863178422647986e+28],
 '7953cef088684d6a82637fe6a9e7e7a8': [99.25535374410566],
 'bf8df4bed5c89ab2aca38cc649b8038e': [193.00221568602112],
 '258c76c5552e3c876c3e24e9658c0205': [134.8482799324594],
 '98e63567a2f4907c9cb2637247651530': [571.3077501288395],
 '5c